# Label Leakage 

Label Leakage (Oscar et al.) is one of the weaknesses in SplitNN, and it means that the intermediate gradient which the server sends to the client may be able to allow the client to extract the private ground-truth labels that the server has. We currently support measuring leak_auc that measures how well the l2 norm of the communicated gradient can predict y by the AUC of the ROC curve. Also, we allow you to avoid this leakage with the defense method called *max norm*.

reference   
https://arxiv.org/abs/2102.08504

In [1]:
from attacksplitnn.splitnn import Client, Server, SplitNN
from attacksplitnn.defense import max_norm
from attacksplitnn.attack import NormAttack
from attacksplitnn.utils import DataSet

In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score

## Set parameters

In [3]:
config = {
    "batch_size":128
}

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Load and preprocess data

we use the ["Credit Card Fraud Detection"](https://www.kaggle.com/mlg-ulb/creditcardfraud) dataset.

reference  
https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [5]:
raw_df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')

In [6]:
raw_df_neg = raw_df[raw_df["Class"] == 0]
raw_df_pos = raw_df[raw_df["Class"] == 1]

down_df_neg = raw_df_neg#.sample(40000)
down_df = pd.concat([down_df_neg, raw_df_pos])

In [7]:
neg, pos = np.bincount(down_df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 284807
    Positive: 492 (0.17% of total)



In [8]:
cleaned_df = down_df.copy()

# You don't want the `Time` column.
cleaned_df.pop('Time')

# The `Amount` column covers a huge range. Convert to log-space.
eps = 0.001 # 0 => 0.1¢
cleaned_df['Log Ammount'] = np.log(cleaned_df.pop('Amount')+eps)

In [9]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [10]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (182276,)
Validation labels shape: (45569,)
Test labels shape: (56962,)
Training features shape: (182276, 29)
Validation features shape: (45569, 29)
Test features shape: (56962, 29)


In [11]:
train_dataset = DataSet(train_features,
                        train_labels.astype(np.float64).reshape(-1, 1))
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config["batch_size"],
                                           shuffle=True)

test_dataset = DataSet(test_features,
                       test_labels.astype(np.float64).reshape(-1, 1))
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=config["batch_size"],
                                          shuffle=True)

## Train SplitNN

In [12]:
hidden_dim = 16

class FirstNet(nn.Module):
    def __init__(self):
        super(FirstNet, self).__init__()        
        self.L1 = nn.Linear(train_features.shape[-1],
                            hidden_dim)

    def forward(self, x):
        x = self.L1(x)
        x = nn.functional.relu(x)
        return x
    
class SecondNet(nn.Module):
    def __init__(self):
        super(SecondNet, self).__init__()        
        self.L2 = nn.Linear(hidden_dim,
                            1)

    def forward(self, x):
        x = self.L2(x)
        x = torch.sigmoid(x)
        return x
    
def torch_auc(label, pred):
    return roc_auc_score(label.detach().numpy(),
                         pred.detach().numpy())

### Vanila SplitNN

You can see that the leak auc is high, and it indicates that the client can correctly predict the label of each data by analyzing the intermediate gradient. 

In [13]:
model_1 = FirstNet()
model_1 = model_1.to(device)

model_2 = SecondNet()
model_2 = model_2.to(device)

model_1.double()
model_2.double()

opt_1 = optim.Adam(model_1.parameters(), lr=1e-3)
opt_2 = optim.Adam(model_2.parameters(), lr=1e-3)

criterion = nn.BCELoss()

client = Client(model_1)
server = Server(model_2)

splitnn = SplitNN(client, server, opt_1, opt_2)

In [14]:
splitnn.train()
for epoch in range(3):
    epoch_loss = 0
    epoch_outputs = []
    epoch_labels = []
    for i, data in enumerate(train_loader):
        splitnn.zero_grads()
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = splitnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        epoch_loss += loss.item() / len(train_loader.dataset)
        
        epoch_outputs.append(outputs)
        epoch_labels.append(labels)
        
        splitnn.backward()
        splitnn.step()
        
    print(epoch_loss, torch_auc(torch.cat(epoch_labels),
                                torch.cat(epoch_outputs)))

0.0006068007277306289 0.8524671216494538
3.438545326414842e-05 0.9359523384907426
2.6487558729677185e-05 0.9665791695222752


In [18]:
nall = NormAttack(splitnn)
nall.attack(train_loader, criterion, device)

0.9995624315496018

### SplitNN with max_norm

You can mitigate this problem with max_norm. To use this defense method, you should create a custom Server class.

In [19]:
class Server_with_max_norm(Server):
    def __init__(self, server_model):
                super().__init__(server_model)
            
    def backward(self):
        grad_to_client = self.intermidiate_to_server.grad.clone()
        self.grad_to_client = max_norm(grad_to_client)
        return self.grad_to_client

In [22]:
model_1 = FirstNet()
model_1 = model_1.to(device)

model_2 = SecondNet()
model_2 = model_2.to(device)

model_1.double()
model_2.double()

opt_1 = optim.Adam(model_1.parameters(), lr=1e-3)
opt_2 = optim.Adam(model_2.parameters(), lr=1e-3)

criterion = nn.BCELoss()

client = Client(model_1)
server = Server_with_max_norm(model_2)

splitnn = SplitNN(client, server, opt_1, opt_2)

In [23]:
splitnn.train()
for epoch in range(10):
    epoch_loss = 0
    epoch_outputs = []
    epoch_labels = []
    for i, data in enumerate(train_loader):
        splitnn.zero_grads()
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = splitnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        epoch_loss += loss.item() / len(train_loader.dataset)
        
        epoch_outputs.append(outputs)
        epoch_labels.append(labels)
        
        splitnn.backward()
        splitnn.step()
        
    print(epoch_loss, torch_auc(torch.cat(epoch_labels),
                                torch.cat(epoch_outputs)))

0.0015609379987302856 0.2095180843780458
0.0004217733845897698 0.09235266566990714
0.00030866593361264933 0.0823424120705995
0.0002723157023194367 0.08050197608698967
0.0001930321600647744 0.14828702490681528
0.00011871603878143286 0.5931839624063057
7.069953111490891e-05 0.8816621361027034
5.495629310577138e-05 0.8999985286155904
4.7659140259583914e-05 0.9045895847965927
4.447669198443257e-05 0.9121939653384378


In [26]:
nall = NormAttack(splitnn)
nall.attack(train_loader, criterion, device)

C:\Users\kanka\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


0.08860336546387786

### SplitNN with dp

In [31]:
from opacus import PrivacyEngine

model_1 = FirstNet()
model_1 = model_1.to(device)

model_2 = SecondNet()
model_2 = model_2.to(device)

model_1.double()
model_2.double()

opt_1 = optim.Adam(model_1.parameters(), lr=1e-3)
opt_2 = optim.Adam(model_2.parameters(), lr=1e-3)

privacy_engine = PrivacyEngine(
    model_2,
    sample_rate=0.01,
    alphas=[10, 100],
    noise_multiplier=1.3,
    max_grad_norm=1.0,
)
privacy_engine.attach(opt_2)

criterion = nn.BCELoss()

client = Client(model_1)
server = Server(model_2)

splitnn = SplitNN(client, server, opt_1, opt_2)

C:\Users\kanka\AppData\Local\Programs\Python\Python39\lib\site-packages\opacus\privacy_engine.py:522: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  warnings.warn(
C:\Users\kanka\AppData\Local\Programs\Python\Python39\lib\site-packages\opacus\privacy_engine.py:194: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(


In [32]:
splitnn.train()
for epoch in range(3):
    epoch_loss = 0
    epoch_outputs = []
    epoch_labels = []
    for i, data in enumerate(train_loader):
        splitnn.zero_grads()
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = splitnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        epoch_loss += loss.item() / len(train_loader.dataset)
        
        epoch_outputs.append(outputs)
        epoch_labels.append(labels)
        
        splitnn.backward()
        splitnn.step()
        
    print(epoch_loss, torch_auc(torch.cat(epoch_labels),
                                torch.cat(epoch_outputs)))

C:\Users\kanka\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


0.0004997869971540773 0.8618391135103936


C:\Users\kanka\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


3.178918249602484e-05 0.9508701200117853


C:\Users\kanka\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


2.556448480792446e-05 0.9726919208232094


In [33]:
nall = NormAttack(splitnn)
nall.attack(train_loader, criterion, device)

C:\Users\kanka\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


0.9997876774569372